In [2]:
"""
Fire occurrence data: FPA-FOD v6

General workflow:

- Tidy the FPA-FOD ignitions data
- Extract the landcover composition within Xm buffer of ignition locations
- Assess the probability of ignition -> large fire growth based on landcover type

Eventually:

Extract variables from gridMET to account for extreme fire weather during the ignition

"""

import os, time
import numpy as np
import pandas as pd
import geopandas as gpd
import rioxarray as rxr
import rasterio as rio
import matplotlib
import matplotlib.pyplot as plt

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'

print("Ready!")

Ready!


In [5]:
fpa = gpd.read_file(os.path.join(maindir,'aspen-fire/Aim2/data/spatial/raw/USFS/FPA_FOD/fpa-fod_west.gpkg'))
fpa = fpa.to_crs(proj)
print(fpa.head())

   OBJECTID  FOD_ID      FPA_ID SOURCE_SYSTEM_TYPE SOURCE_SYSTEM  \
0     33335   33504  FS-1474237                FED   FS-FIRESTAT   
1    224459  227966     W-28574                FED      DOI-WFMI   
2    224375  227881     W-38733                FED      DOI-WFMI   
3    329334  336835    W-125187                FED      DOI-WFMI   
4    224642  228152     W-16575                FED      DOI-WFMI   

  NWCG_REPORTING_AGENCY NWCG_REPORTING_UNIT_ID  \
0                    FS                USIDCTF   
1                   BLM                USIDIFD   
2                   BLM                USIDIFD   
3                   BLM                USIDIFD   
4                   BLM                USIDIFD   

          NWCG_REPORTING_UNIT_NAME SOURCE_REPORTING_UNIT  \
0  Caribou-Targhee National Forest                  0415   
1             Idaho Falls District                 IDIFD   
2             Idaho Falls District                 IDIFD   
3             Idaho Falls District                

## Calculate the landcover type at point of ignition

As a first step, we can identify the landcover type at the point of ignition. Likely, we will want to also use a buffered region around the ignition location and assess the vegetation composition to get a better understanding of the distribution of vegetation types near the ignition source.

In [ ]:
# Define a function for point-based raster sampling
def img_vals_at_pts(img, points, band_names):
    coord_list = [(x, y) for x, y in zip(points["geometry"].x, points["geometry"].y)]
    n_bands = img.count
    print(f"Number of bands to process: {n_bands}")
    for i in range(0,n_bands):
        band = desc[i]
        print(str(i)+'_'+band)
        points[f"{band}"] = [x for x in img.sample(coord_list, indexes=i+1)]
    points_df = points.reset_index()
    points_df[desc] = points_df[band_names].astype(np.float32)
    return points_df